In [1]:
import cdsapi
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
years = np.arange(1979, 2025, 1)
title = 'ERA5_reanalysis_western_NA'

In [3]:
def get_ERA5_data(year, title):   
    # Pull ERA5 data from the CDS API client
    if not os.path.exists(f'/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc'):
        dataset = "reanalysis-era5-pressure-levels"
        request = {
            "product_type": ["reanalysis"],
            "variable": [
                "geopotential",
                "specific_humidity",
                "temperature",
                "u_component_of_wind",
                "v_component_of_wind",
                "vorticity"
            ],
            "year": [str(year)],
            "month": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12"
            ],
            "day": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12",
                "13", "14", "15",
                "16", "17", "18",
                "19", "20", "21",
                "22", "23", "24",
                "25", "26", "27",
                "28", "29", "30",
                "31"
            ],
            "time": [
                "00:00", "01:00", "02:00",
                "03:00", "04:00", "05:00",
                "06:00", "07:00", "08:00",
                "09:00", "10:00", "11:00",
                "12:00", "13:00", "14:00",
                "15:00", "16:00", "17:00",
                "18:00", "19:00", "20:00",
                "21:00", "22:00", "23:00"
            ],
            "pressure_level": ["500"],
            "data_format": "netcdf",
            "area": [45, -110, 35, -100]
        }
        target = f'/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc'
        client = cdsapi.Client()
        client.retrieve(dataset, request, target=target)
        print(f'ERA5 data for {year} has been downloaded.')
        # print the approximate size of the file
        print(f'The file size is {os.path.getsize(f"/storage/dlhogan/data/raw_data/{title}_{year}_v2.nc") / 1e6} MB')
        return 
    else:
        print(f'ERA5 data for {year} has already been downloaded.')
    return

In [4]:
for year in years:
    get_ERA5_data(year, title)

ERA5 data for 1979 has already been downloaded.
ERA5 data for 1980 has already been downloaded.
ERA5 data for 1981 has already been downloaded.
ERA5 data for 1982 has already been downloaded.
ERA5 data for 1983 has already been downloaded.
ERA5 data for 1984 has already been downloaded.
ERA5 data for 1985 has already been downloaded.
ERA5 data for 1986 has already been downloaded.
ERA5 data for 1987 has already been downloaded.
ERA5 data for 1988 has already been downloaded.
ERA5 data for 1989 has already been downloaded.
ERA5 data for 1990 has already been downloaded.
ERA5 data for 1991 has already been downloaded.
ERA5 data for 1992 has already been downloaded.
ERA5 data for 1993 has already been downloaded.
ERA5 data for 1994 has already been downloaded.
ERA5 data for 1995 has already been downloaded.
ERA5 data for 1996 has already been downloaded.
ERA5 data for 1997 has already been downloaded.
ERA5 data for 1998 has already been downloaded.
ERA5 data for 1999 has already been down

In [ ]:
#! /usr/bin/env python

from mpl_toolkits.basemap import Basemap, shiftgrid, cm
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime

datetime_origin = datetime.datetime(1, 1, 1, 0, 0, 0, 0)
# In this case, this is 500 mb. Can check below with d.variables["level"][:]
level = 5
time = 0
# 277830 is because this is a ~2.5 x 2.5 degree grid; 111132 meters are in about 111 km appart. This is fairly rough.
meters_per_grid = 277830

# Download the dataset from: ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc
d = Dataset("hgt.mon.mean.nc")
hgt = d.variables["hgt"][:][time, level, :, :]  # Not sure about the dims, here.
lon = d.variables["lon"][:]
# need to reverse direction of lat dimension so it's increasing.
lat = d.variables["lat"][:][::-1]
hgt = hgt[::-1, :]
hgt, lon = shiftgrid(180, hgt, lon, start = False)
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# Lambert Azimuthal Equal Area
# Larger area:
#m = Basemap(width = 12000000, height = 12000000, resolution = "l", projection = "laea", lat_ts = 50, lat_0 = 50, lon_0 = -107.0)
# Smaller area:
#m = Basemap(width = 12000000, height = 8000000, resolution = "l", projection = "laea", lat_ts = 50, lat_0 = 50, lon_0 = -107.0)
# For north Polar Stereographic projection
m = Basemap(projection='npstere',boundinglat=10,lon_0=270,resolution='l')
nx = int((m.xmax - m.xmin)/meters_per_grid); ny = int((m.ymax - m.ymin)/meters_per_grid)
hgt = m.transform_scalar(hgt, lon, lat, nx, ny)
im = m.imshow(hgt, interpolation = "none")

m.drawcoastlines()
parallels = np.arange(-90, 90, 30)
meridians = np.arange(-180, 180, 60)
m.drawparallels(parallels, labels = [1, 0, 0, 1])
m.drawmeridians(meridians, labels = [1, 0, 0, 1])

cb = m.colorbar(im, "right", size = "5%", pad = "2%")
ax.set_title("500 mb Geopotential Height")
plt.show()